# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [30]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [31]:
# Output File (CSV)
citi_data_df = pd.read_csv("../WeatherPy/city_data.csv")
citi_data_df

Unnamed: 0         city country      lat       lon  max_temp  humidity  \
0             0      rikitea      PF -23.1203 -134.9692     79.02        70   
1             1     phangnga      TH   8.4509   98.5298     80.60        78   
2             2  muyezerskiy      RU  63.9333   31.6500     -2.25        91   
3             3      kodinsk      RU  58.6881   99.1844     -4.72        89   
4             4  tuktoyaktuk      CA  69.4541 -133.0374    -38.20        67   
..          ...          ...     ...      ...       ...       ...       ...   
550         550     hokitika      NZ -42.7167  170.9667     58.98        73   
551         551    las vegas      US  36.1750 -115.1372     62.60        20   
552         552   hudiksvall      SE  61.7290   17.1036     18.00        89   
553         553     zyryanka      RU  65.7500  150.8500    -13.54        87   
554         554     yancheng      CN  33.3856  120.1253     46.92        69   

     wind_speed        date  cloudiness  
0          7.02  1612920182          16  
1          6.91  1612920738          20  
2          3.53  1612920738          90  
3          4.92  1612920738          68  
4          4.61  1612920180          20  
..          ...         ...         ...  
550       11.16  1612920206          26  
551        6.91  1612920562           1  
552        7.31  1612920770         100  
553        9.24  1612920213           1  
554        6.33  1612920770         100  

[555 rows x 10 columns]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [32]:
#Configure gmaps.
gmaps.configure(api_key = g_key)

# Store latitude and longitude in locations
citi_data_df = citi_data_df.dropna()
locations = citi_data_df[["lat", "lon"]].astype(float)

#Humidity as the weight.
weight = citi_data_df["humidity"].astype(float)

#Add Heatmap layer to map.
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, 
                                 max_intensity=300, point_radius=5)
fig.add_layer(heat_layer)
                                 

                                 
fig

                                
                                

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
#narrrow to a max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
# Zero cloudiness.
#Drop any rows will null values.

narrow_city_df = citi_data_df.loc[(citi_data_df["max_temp"] > 70) & (citi_data_df["max_temp"] < 80) & (citi_data_df["cloudiness"] == 0), :]
narrow_city_df = narrow_city_df.dropna(how='any')
narrow_city_df.reset_index(inplace=True)
del narrow_city_df['index']
narrow_city_df.head()



Unnamed: 0             city country      lat       lon  max_temp  humidity  \
0          17             buin      CL -33.7333  -70.7500     78.01        60   
1          46      east london      ZA -33.0153   27.9116     71.06        85   
2          50  arraial do cabo      BR -22.9661  -42.0278     74.79        85   
3          78        carnarvon      AU -24.8667  113.6333     75.20        83   
4          81   saint-philippe      RE -21.3585   55.7679     77.00        65   

   wind_speed        date  cloudiness  
0       12.66  1612920739           0  
1        2.08  1612920186           0  
2        2.53  1612920192           0  
3       27.63  1612920184           0  
4        5.75  1612920018           0

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df = narrow_city_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["lat"]
    lng = row["lon"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    import json 
    
    hotel_name = hotel_name.json()
#     print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
#     # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df.head()





Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


Unnamed: 0             city country      lat       lon  max_temp  humidity  \
0          17             buin      CL -33.7333  -70.7500     78.01        60   
1          46      east london      ZA -33.0153   27.9116     71.06        85   
2          50  arraial do cabo      BR -22.9661  -42.0278     74.79        85   
3          78        carnarvon      AU -24.8667  113.6333     75.20        83   
4          81   saint-philippe      RE -21.3585   55.7679     77.00        65   

   wind_speed        date  cloudiness                       Hotel Name  
0       12.66  1612920739           0                          Paihuen  
1        2.08  1612920186           0                Blue Lagoon Hotel  
2        2.53  1612920192           0          Orlanova Boutique Hotel  
3       27.63  1612920184           0  Carnarvon Sea Change Apartments  
4        5.75  1612920018           0             Les Embruns Du Baril

In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [36]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer (locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# Display figure
fig


Figure(layout=FigureLayout(height='420px'))